# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "../output_data/cities.csv"

cities_df = pd.read_csv(file)
cities_df.head()


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Longyearbyen,78.22,15.64,19.40,79,20,8.05,SJ,1602971102
1,Itarema,-2.92,-39.92,79.39,78,81,15.48,BR,1602971121
2,New Norfolk,-42.78,147.06,46.99,91,55,1.01,AU,1602971121
3,Rikitea,-23.12,-134.97,74.21,80,100,20.98,PF,1602971121
4,Barrow,71.29,-156.79,17.60,78,1,12.75,US,1602970832


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df["Humidity"].astype(float)

gmaps.configure(api_key= g_key)
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [3]:
# use conditionals to narrow down weather data
ideal_weather_df = cities_df.loc[(cities_df['Max Temp'] < 85) & (cities_df['Max Temp'] > 75) & (cities_df['Humidity'] < 35) &(cities_df['Wind Speed'] < 10)] 

ideal_weather_df.to_csv("../output_data/ideal_weather.csv", index=False)




In [4]:
file2 = "../output_data/ideal_weather.csv"

ideal_weather = pd.read_csv(file2)
ideal_weather.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Gouré,13.98,10.27,84.99,32,0,5.82,NE,1602971123
1,Jalu,29.03,21.55,81.84,22,0,8.52,LY,1602971170
2,Makkah al Mukarramah,21.43,39.83,78.67,32,0,4.50,SA,1602971147
3,Ubari,26.59,12.78,77.18,18,6,1.12,LY,1602971199
4,Aswān,24.09,32.91,82.06,28,0,8.57,EG,1602971199


In [5]:
ideal_weather["Hotel Name"] = ""

ideal_weather

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Gouré,13.98,10.27,84.99,32,0,5.82,NE,1602971123,
1,Jalu,29.03,21.55,81.84,22,0,8.52,LY,1602971170,
2,Makkah al Mukarramah,21.43,39.83,78.67,32,0,4.50,SA,1602971147,
3,Ubari,26.59,12.78,77.18,18,6,1.12,LY,1602971199,
4,Aswān,24.09,32.91,82.06,28,0,8.57,EG,1602971199,
5,Bandar-e Lengeh,26.56,54.88,78.80,34,0,9.17,IR,1602971200,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel = []

# params 
params = {
    "radius": 50000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify nearby hotels
for index, row in ideal_weather.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration but keep params the same
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Lodging" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    

    # convert to json
    name_address = name_address.json()

    # add try-except to skip any that are missing a data point.
    try:
        ideal_weather.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError, NameError):
        print("Missing field/result... skipping.")

hotel_df = pd.DataFrame(ideal_weather)        
        
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Gouré,13.98,10.27,84.99,32,0,5.82,NE,1602971123,Inspection de l'Enseignement Primaire de Kellé
1,Jalu,29.03,21.55,81.84,22,0,8.52,LY,1602971170,Jalu Hotel
2,Makkah al Mukarramah,21.43,39.83,78.67,32,0,4.50,SA,1602971147,Hotel Fairmont Makkah Clock Royal Tower
3,Ubari,26.59,12.78,77.18,18,6,1.12,LY,1602971199,فندق أوباري
4,Aswān,24.09,32.91,82.06,28,0,8.57,EG,1602971199,Nuba Nile
5,Bandar-e Lengeh,26.56,54.88,78.80,34,0,9.17,IR,1602971200,kong Hotel


In [8]:
#plot markers for nearby hotels
gmaps.configure(api_key= g_key)

marker_locations = ideal_weather[["Latitude", "Longitude"]]
    

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather.iterrows()]
locations = ideal_weather[["Latitude", "Longitude"]]



In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))